<img src='./img/egu21_logo.png' alt='Logo EU Copernicus EUMETSAT' align='left' width='25%'></img><img src='./img/atmos_logos.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='65%'></img></span>

<br>

<br>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./05_Metop-ABC_GOME-2_AAI_L3_load_browse.ipynb"><< 05 - Metop-ABC GOME-2 AAI Level 3</a><span style="float:right;"><a href="./13_CAMS_European_air_quality_forecast_dust_concentration_exercise.ipynb">13 - CAMS European air quality forecast - Dust - Exercise >></a></span>

<div class="alert alert-block alert-info">
<b>EXERCISE WORKBOOK</b>
</div>

<hr>

# CAMS - Global Near-Real-Time Forecast - Dust Aerosol Optical Depth

This notebook is an `exercise workbook` which allows you to practise the preparation of CAMS global near-real-time forecast data, using the variable `Dust Aerosol Optical Depth` to analyse a dust event at the beginning of February 2021.

### How it works

This exercise workbook consists of two types of exercises:

#### Coding assignments
Coding assignments ask you to fill an empty code cell with code.
You recognise `coding assignments` as the yellow-coloured boxes.

<div class="alert alert-block alert-warning">
<b>Coding assignment</b>
</div>

#### Questions

Questions ask you to reflect on a result and output. `Questions` are provided as green-coloured boxes.

<div class="alert alert-block alert-success">
<b>Question</b>
</div>

#### Outline:
* [1 - Load and browse dust aerosol optical depth (duaod) at 550nm of the CAMS global near-real-time forecast](#load_browse_duaod)
* [2 - Retrieve the data variable dust AOD at 550nm as xarray.DataArray](#data_retrieve_duaod)
* [3 - Visualize dust AOD at 550nm](#visualize_duaod)
* [4 - Create a geographical subset for Europe](#subset_duaod)

<hr>

##### Load required libraries

In [57]:
%matplotlib inline
import os
import xarray as xr
import numpy as np
import netCDF4 as nc
import pandas as pd

from IPython.display import HTML

import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.cm import get_cmap
from matplotlib import animation
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh
from datetime import datetime

import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)

##### Load helper functions

In [58]:
%run ./functions.ipynb

<hr>

### <a id='load_browse_duaod'></a>Load and browse `CAMS global near-real-time forecast` data

CAMS global near-real-time forecast data is available either in `GRIB` or `netCDF`. The data for the present example has been downloaded as `netCDF`. See an example how to download the data with the `ECMWF WebAPI` [here](./01_overview_atmospheric_composition_data_retrieve.ipynb#ecmwf_archive).

You can use xarray's function `xr.open_dataset()` to open the netCDF file as `xarray.Dataset`.

In [59]:
file = xr.open_dataset('./eodata/cams/near_real_time/2021/02/05/20210205-10_dustAOD.nc')
file



<xarray.Dataset>
Dimensions:    (latitude: 451, longitude: 900, time: 11)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.6 -179.2 ... 178.8 179.2 179.6
  * latitude   (latitude) float32 90.0 89.6 89.2 88.8 ... -89.2 -89.6 -90.0
  * time       (time) datetime64[ns] 2021-02-05 ... 2021-02-10
Data variables:
    duaod550   (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2021-04-23 10:00:25 GMT by grib_to_netcdf-2.21.0: grib_to_n...

<div class="alert alert-block alert-success">
<b>Question:</b><br>

Inspect the loaded `xarray.Dataset`:
- How many dimensions does the data array have?
- What data variables does the xarray.Dataset offer?
</div>

<div class="alert alert-block alert-warning">
<b>Coding assignment:</b><br>
   Enter below the code which selects the coordinate <b>time</b> from the xarray.Dataset <i>file</i>.
</div>

In [ ]:
# Enter your solution here








<br>

<div class="alert alert-block alert-success">
<b>Question:</b><br>
    
How many hours does each timestep have in the loaded `xarray.DataArray`?
</div>

### <a id='shift_duaod'></a>Bring longitude coordinates onto a [-180,180] grid

You can assign new values to coordinates in an `xarray.Dataset`. You can do so with the `assign_coords()` function, which you can apply onto a `xarray.Dataset`. With the code below, you shift your longitude grid from [0,360] to [-180,180]. At the end, you sort the longitude values in an ascending order.

In [28]:
file_assigned = file.assign_coords(longitude=(((file.longitude + 180) % 360) - 180)).sortby('longitude')
file_assigned

<xarray.Dataset>
Dimensions:    (latitude: 451, longitude: 900, time: 11)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.6 -179.2 ... 179.20001 179.6
  * latitude   (latitude) float32 90.0 89.6 89.2 88.8 ... -89.2 -89.6 -90.0
  * time       (time) datetime64[ns] 2021-02-05 ... 2021-02-10
Data variables:
    duaod550   (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2021-04-23 10:00:25 GMT by grib_to_netcdf-2.21.0: grib_to_n...

<br>

A quick check of the longitude coordinates of the new `xarray.Dataset` shows you that the longitude values range now between [-180, 180].

In [29]:
file_assigned.longitude

<xarray.DataArray 'longitude' (longitude: 900)>
array([-180.     , -179.6    , -179.2    , ...,  178.79999,  179.20001,
        179.6    ], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.6 -179.2 ... 179.20001 179.6

### <a id='data_retrieve_duaod'></a>Retrieve the variable `Dust Aerosol Optical Depth at 550nm` as xarray.DataArray

Let us store the data variable `Dust Aerosol Optical Depth (AOD) at 550nm` as `xarray.DataArray` with the name `du_aod`.

<div class="alert alert-block alert-warning">
<b>Coding assignment:</b><br>
   Enter below the code which selects the data variable <b>duaod550</b> from the xarray.Dataset <i>file_assigned</i> and call it <b>du_aod</b> 
</div>

In [30]:
# Enter your solution here








Above, you see that the variable `du_aod` has two attributes, `units` and `long_name`. Let us define variables for those attributes. The variables can be used for visualizing the data.

<div class="alert alert-block alert-warning">
<b>Coding assignment:</b><br>
   Enter below the code which selects the attribute <b>long_name</b> from the xarray.Dataset <i>du_aod</i> and call it <b>long_name</b>. Then, select the attribute <b>units</b> from the xarray.Dataset <i>du_aod</i> and call it <b>units</b>.
</div>

In [30]:
# Enter your solution here








Let us do the same for the coordinates `longitude` and `latitude`.

<div class="alert alert-block alert-warning">
<b>Coding assignment:</b><br>
   Enter below the code which selects the attribute <b>latitude</b> from the xarray.Dataset <i>du_aod</i> and call it <b>latitude</b>. Then, select the attribute <b>longitude</b> from the xarray.Dataset <i>du_aod</i> and call it <b>longitude</b>.
</div>

In [32]:
# Enter your solution here








<br>

### <a id='visualize_duaod'></a>Visualize `Dust Aerosol Optical Depth at 550nm`

The next step is to visualize the Dust Aerosol Optical Depth data for one time step.

<div class="alert alert-block alert-warning">
<b>Coding assignment:</b><br>

Make use of the function [visualize_pcolormesh](../functions.ipynb#visualize_pcolormesh) in order to visualize the xarray.DataArray <code>du_aod</code>. <br>
    
The following kwargs have to be defined:
* `data_array`
* `longitude`
* `latitude`
* `projection`
* `color_scale`
* `unit`
* `longname`
* `vmin`,
* `vmax`,
* `lonmin`, `lonmax`, `latmin`, `latmax`
* `set_global`
    
**HINT**: 
* With `?visualize_pcolormesh` you can open the function's docstring to see what keyword arguments are needed to prepare your plot
* You can use `YlGn` as color map, `ccrs.PlateCarree()` as projection
* Try to add the time information as part of the title. Add the string of the datetime information to the `longname` variable: `long_name + ' on ' +str(du_aod.time[0].dt.strftime('%Y-%m-%d').data)`.
* Try using `0` for the vmin and `1.5` for the vmax. 

</div>

In [42]:
# Enter your solution here








<br>

### <a id='subset_duaod'></a>Create a geographical subset for Europe

The map above shows organic matter of Dust Aerosol Optical Depth at 550nm globally. Let us create a geographical subset for Europe, in order to better analyse the Saharan dust event over Europe.

<div class="alert alert-block alert-warning">
<b>Coding assignment:</b><br>

Create a geographical subset for Europe. Make use of the function <a href='../functions.ipynb#generate_geographical_subset'>generate_geographical_subset</a> and call the new DataArray <code>du_aod_subset</code>.<br>

The bounding box information can be the following:<br>
- **latmin=28.**
- **latmax=71.**
- **lonmin=-22.**
- **lonmax=43**

<b>HINT:</b><br> With `?generate_geographical_subset`, you can see what keyword arguments the function requires.

</div>

In [50]:
# Enter your solution here








<div class="alert alert-block alert-success">
<b>Question:</b><br>

What is the largest `latitude` coordinate value for DataArray **du_aod_subset** compared to that of DataArray **du_aod**?
</div>

<br>

Let us now visualize the subsetted `xarray.DataArray` over Europe.

<div class="alert alert-block alert-warning">
<b>Coding assignment:</b><br>

Make use of the function [visualize_pcolormesh](../functions.ipynb#visualize_pcolormesh) in order to visualize the subsetted DataArray <code>du_aod_subset</code>. <br>
    
Make sure to set the `set_global` kwarg to `False` and specify the latitude and longitude bounding box to the ones specified for Europe.

Try also to add the time information as part of the title. Add the string of the datetime information to the `longname` variable: `long_name + ' on ' +str(du_aod_subset.time[0].dt.strftime('%Y-%m-%d').data)`.

In [52]:
# Enter your solution here








<br>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./05_Metop-ABC_GOME-2_AAI_L3_load_browse.ipynb"><< 05 - Metop-ABC GOME-2 AAI Level 3</a><span style="float:right;"><a href="./13_CAMS_European_air_quality_forecast_dust_concentration_exercise.ipynb">13 - CAMS European air quality forecast - Dust - Exercise >></a></span>

<hr>

<img src='../../img/copernicus_logo.png' alt='Logo EU Copernicus' align='right' width='20%'><br><br><br><br>
<p style="text-align:right;">This project is licensed under the <a href="./LICENSE">MIT License</a> and is developed under a Copernicus contract.